In [1]:
import pandas as pd
import polars as pl

In [2]:
%ls /home/jovyan/data/

'Cuenta Corriente DOLARES CABLE 30-06-25.xlsx'*
'Cuenta Corriente PESOS 30-03-25.xlsx'*
 cuenta_corriente_pesos_30_06_25.csv
'Cuenta Corriente PESOS 30-06-25.xlsx'*
 operaciones_filtradas.csv


In [3]:
df_cuenta_corriente = pd.read_excel(
    "/home/jovyan/data/Cuenta Corriente PESOS 30-06-25.xlsx",
    sheet_name="Cuenta Corriente PESOS 30-06-25"
)

df_cuenta_corriente.to_csv("/home/jovyan/data/cuenta_corriente_pesos_30_06_25.csv", index=False)

In [4]:
df_cuenta_corriente

,Liquida,Operado,Comprobante,Numero,Cantidad,Especie,Precio,Importe,Saldo,Referencia
0,2024-06-28,2024-06-28,RETENCION,634586,0.00,NaN,0.000000,-76.29,577.81,GGAL RET
1,2024-06-28,2024-06-28,DIVIDENDOS,618806,0.00,GGAL,0.000000,1075.41,1653.22,GGAL BYMA
2,2024-07-01,2024-06-28,LIQUIDACION RESCATE FCI,263552,-3852.56,BMACTAA,9.820525,37834.16,39487.38,NaN
3,2024-07-01,2024-07-01,ORDEN DE PAGO,604216,0.00,NaN,0.000000,-39000.00,487.38,TRANSFERENCIA VIA MEP
4,2024-07-05,2024-07-05,RECIBO DE COBRO,1033526,0.00,NaN,0.000000,300000.00,300487.38,CREDITO CTA. CTE.
...,...,...,...,...,...,...,...,...,...,...
200,2025-06-24,2025-06-24,ORDEN DE PAGO,456588,0.00,NaN,0.000000,-50000.00,1603509.35,TRANSFERENCIA VIA MEP
201,2025-06-24,2025-06-24,ORDEN DE PAGO,457132,0.00,NaN,0.000000,-130000.00,1473509.35,TRANSFERENCIA VIA MEP
202,2025-06-27,2025-06-27,RECIBO DE COBRO,667219,0.00,NaN,0.000000,208147.52,1681656.87,CREDITO CTA. CTE.
203,2025-06-30,2025-06-27,COMPRA NORMAL,4977570,12.00,YPFD,40159.720000,-481916.64,1199740.23,NaN


In [5]:
importe = 10*5+5*4.9
cantidad = 10+5
precio_promedio_de_comra = importe / cantidad

round(precio_promedio_de_comra,2)

4.97

In [15]:
import pandas as pd

def calcular_ganancia_perdida(ruta_archivo_csv: str):
    """
    Calcula la ganancia o pérdida realizada para cada operación de venta
    de activos a partir de un archivo CSV de cuenta corriente.

    Args:
        ruta_archivo_csv (str): La ruta al archivo CSV exportado del broker.

    Returns:
        pandas.DataFrame: Un DataFrame con el resultado de cada venta.
    """
    # 1. Cargar y preparar los datos del CSV
    df = pd.read_csv(
        ruta_archivo_csv,
        delimiter=',',  # El delimitador parece ser punto y coma
        decimal='.'     # Usar coma como separador decimal
    )

    # Convertir columnas a los tipos de datos correctos
    df['Operado'] = pd.to_datetime(df['Operado'], dayfirst=True)
    # Asegurarse que las columnas numéricas sean floats
    for col in ['Cantidad', 'Precio', 'Importe']:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

    # 2. Filtrar solo operaciones de compra y venta
    operaciones = df[df['Comprobante'].isin(['COMPRA NORMAL', 'VENTA'])].copy()
    operaciones = operaciones.sort_values(by='Operado', ascending=True)

    # 3. Lógica principal: Iterar y calcular
    cartera = {}  # Diccionario para seguir el estado de cada activo
                    # Ejemplo: {'GGAL': {'cantidad': 100, 'costo_total': 45000}}
    resultados = [] # Lista para guardar los resultados de las ventas

    print("Procesando operaciones...")

    for index, op in operaciones.iterrows():
        especie = op['Especie']
        cantidad = op['Cantidad']
        importe = op['Importe']
        precio_op = op['Precio']

        # Inicializar el activo en la cartera si no existe
        if especie not in cartera and op['Comprobante'] == 'VENTA':
            print(f"ADVERTENCIA: Se intentó vender {cantidad} de {especie}, pero no hay registro de compra. Se omitirá.")
            continue

        elif especie not in cartera:
            cartera[especie] = {'cantidad_total': 0.0, 'costo_total': 0.0}

        # Si es una COMPRA
        if op['Comprobante'] == 'COMPRA NORMAL':
            cartera[especie]['cantidad_total'] += abs(cantidad)
            cartera[especie]['costo_total'] += abs(importe) # El importe de compra es negativo
            print(f"Compra: {cantidad:.2f} de {especie} a ${precio_op:.2f}")


        # Si es una VENTA
        elif op['Comprobante'] == 'VENTA':
            if abs(cartera[especie]['cantidad_total']) < abs(cantidad):
                print(f"ADVERTENCIA: Se intentó vender {cantidad} de {especie}, pero solo hay {cartera[especie]['cantidad_total']} en cartera. Se omitirá.")
                cartera.pop(especie, None)
                print(f"Se eliminó {especie} de la cartera por falta de cantidad suficiente.")
                continue

            # Calcular el Precio Promedio de Compra (PPC) al momento de la venta
            if cartera[especie]['cantidad_total'] > 0:
                ppc = cartera[especie]['costo_total'] / cartera[especie]['cantidad_total']
            else:
                ppc = 0 # Evitar división por cero

            # Calcular el costo de los activos vendidos
            costo_de_venta = ppc * cantidad
            # La ganancia es el importe de la venta (positivo) menos el costo
            ganancia_perdida = abs(importe) - abs(costo_de_venta)

            print(f"Venta: {cantidad:.2f} de {especie} a ${precio_op:.2f}. PPC: ${ppc:.2f}. Resultado: ${ganancia_perdida:.2f}")

            # Registrar el resultado de la operación
            resultados.append({
                'Fecha Venta': op['Operado'].date(),
                'Activo': especie,
                'Cantidad Vendida': cantidad,
                'Precio Venta': precio_op,
                'Precio Promedio Compra (PPC)': ppc,
                'Costo Total Venta': abs(costo_de_venta),
                'Ganancia/Perdida ($)': ganancia_perdida
            })

            # Actualizar la cartera después de la venta
            cartera[especie]['cantidad_total'] -= cantidad
            cartera[especie]['costo_total'] -= costo_de_venta

            # Si se vendió todo, se puede resetear el costo para evitar errores de flotantes
            if cartera[especie]['cantidad_total'] < 1e-9: # Un número muy pequeño
                cartera[especie]['cantidad_total'] = 0
                cartera[especie]['costo_total'] = 0


    print("\n¡Cálculo finalizado!")
    return pd.DataFrame(resultados)

# --- EJECUCIÓN DEL SCRIPT ---
# Reemplaza 'ruta/a/tu/archivo.csv' con la ruta real de tu archivo
# Como subiste el archivo, podemos usar su nombre directamente.
nombre_archivo = '/home/jovyan/data/cuenta_corriente_pesos_30_06_25.csv'
resultados_df = calcular_ganancia_perdida(nombre_archivo)

# Mostrar el DataFrame con los resultados
if isinstance(resultados_df, pd.DataFrame) and not resultados_df.empty:
    print("\n--- Resumen de Ganancias y Pérdidas Realizadas ---")
    # Formatear los números para mejor lectura
    pd.options.display.float_format = '{:,.2f}'.format
    print(resultados_df.to_string()) # Usamos to_string para ver todas las filas y columnas
else:
    print("\nNo se encontraron operaciones de venta para calcular o hubo un error.")
    if not isinstance(resultados_df, pd.DataFrame):
        print(resultados_df) # Muestra el mensaje de error

Procesando operaciones...
Compra: 15.00 de AUSO a $3121.76
Compra: 14.00 de CEPU a $1283.95
Compra: 85.00 de FIPL a $371.09
Compra: 12.00 de PAMP a $2613.21
Compra: 10.00 de GGAL a $4113.67
Compra: 25.00 de TRAN a $1932.27
Compra: 5.00 de PAAS a $10422.64
Compra: 5.00 de CVH a $5236.49
Compra: 15.00 de DGCU2 a $1329.26
ADVERTENCIA: Se intentó vender -16.0 de CVH, pero solo hay 5.0 en cartera. Se omitirá.
Se eliminó CVH de la cartera por falta de cantidad suficiente.
Compra: 5.00 de AUSO a $2849.86
Compra: 20.00 de CEPU a $1137.93
Compra: 10.00 de TRAN a $1807.60
Compra: 12.00 de DGCU2 a $1319.19
Compra: 50.00 de FIPL a $309.15
Compra: 1.00 de MIRG a $19259.23
Compra: 48.00 de TECO2 a $1792.49
Compra: 5.00 de CEPU a $1278.91
Compra: 26.00 de AUSO a $3056.30
Compra: 35.00 de DGCU2 a $1419.90
Compra: 150.00 de FIPL a $327.28
Compra: 7.00 de GGAL a $5599.02
Compra: 3.00 de MIRG a $21600.54
Compra: 7.00 de PAAS a $8912.11
Compra: 17.00 de PAMP a $2970.70
Compra: 23.00 de TECO2 a $2039.21
Co

/tmp/ipykernel_123/1697008932.py:22: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['Operado'] = pd.to_datetime(df['Operado'], dayfirst=True)


In [17]:
resultados_df.to_csv('/home/jovyan/data/resultados_ganancias_perdidas.csv', index=False)